In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
all_states = ['ACT', 'NSW', 'NT', 'QLD', 'SA', 'TAS', 'VIC', 'WA']

In [3]:
df1 = pd.read_csv('data/postcodes/australian_postcodes.csv', dtype={'locality': str, 'state': str, 'postcode': str})
df2 = pd.read_csv('data/postcodes/postcode-dataout.txt', header=None, names=['suburb', 'state', 'postcode'], dtype={'suburb': str, 'state': str, 'postcode': str})
df1['locality'] = df1['locality'].str.upper()
df1['state'] = df1['state'].str.upper()
df2['suburb'] = df2['suburb'].str.upper()
df2['state'] = df2['state'].str.upper()

In [4]:
df_diff = pd.merge(df1, df2, left_on=['locality', 'state', 'postcode'], right_on=['suburb', 'state', 'postcode'], how='right')[['suburb', 'state', 'postcode', 'locality']]

df_diff = df_diff[df_diff['locality'].isna()].reset_index(drop=True).sort_values(by=['state', 'suburb'])

df1g = df1.groupby(['locality', 'state']).agg({'postcode': lambda x: ', '.join(x)}).reset_index()

df2g = df2.dropna(inplace=False).groupby(['suburb', 'state']).agg({'postcode': lambda x: ', '.join(x)}).reset_index()

df_diff = pd.merge(df_diff, df1g, left_on=['suburb', 'state'], right_on=['locality', 'state'], how='left', suffixes=('', '_1'))[['suburb', 'state', 'postcode', 'locality', 'postcode_1']]

df_diff = pd.merge(df_diff, df2g, left_on=['suburb', 'state'], right_on=['suburb', 'state'], how='left', suffixes=('', '_2'))[['suburb', 'state', 'postcode', 'locality', 'postcode_1', 'postcode_2']]

df_diff.fillna('', inplace=False)

,suburb,state,postcode,locality,postcode_1,postcode_2
0,&,5585–5831,,,,
1,ABERDEEN,NSW,2359,,2336,"2336, 2359"
2,PENROSE,NSW,2530,,2579,"2530, 2579"
3,THIRLDENE,NSW,2347,,,2347
4,GILLA,QLD,4306,,4314,4306
5,LOCKHART,QLD,4892,,4871,4892
6,MISTAKE CREEK,QLD,4721,,,4721
7,DERRIMUT,VIC,3030,,3026,3030
8,700–762 OSBORNE PARK,WA,6917,,,6917
9,COOKERNUP,WA,6220,,6219,6220


Main data source: https://auspost.com.au/postcode
Verdict: df1 is more accurate than df2.